# (QLora) Fine-tuning Mistral-7b-Instruct to Respond to YouTube Comments

Code authored by: Shaw Talebi <br>
Video link: https://youtu.be/XpoKB3usmKc <br>
Blog link: https://medium.com/towards-data-science/qlora-how-to-fine-tune-an-llm-on-a-single-gpu-4e44d6b5be32 <br>

Colab link: https://colab.research.google.com/drive/1AErkPgDderPW0dgE230OOjEysd0QV1sR?usp=sharing

### imports

In [1]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 63.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 83.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.7/424.7 kB 11.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.5 MB/s eta 0:00:0000:0100:01


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

2024-07-22 09:44:17.887345: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 09:44:17.887491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 09:44:18.007731: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Disable wandb
import os
os.environ['WANDB_MODE'] = 'disabled'

### Load model

In [4]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### Load tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Using Base Model

In [6]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
question = "Tell me about white wine?"
prompt=f'''[INST] {question} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST] Tell me about white wine? [/INST] White wine is a type of wine that is produced from grapes with green or yellow skin. The color of the wine results from the absence of skin contact during the fermentation process. White wine can be made from a wide variety of grape varieties, each imparting unique flavors and aromas to the final product. Some common white grape varieties include Chardonnay, Sauvignon Blanc, Riesling, Pinot Grigio, and Chenin Blanc.

White wine can range in style from crisp and refreshing, with high acidity and fruity flavors, to rich and full-bodied, with complex flavors


#### Prompt Engineering

In [7]:
intstructions_string = f"""You are a helpful and smart chatbot designed to assist customers of our wine-selling business by providing quick, accurate, and relevant answers to user's query.\
Your primary goal is to help users find information about our wine collection, make recommendations, answer questions about orders and shipping, and provide details on whatever question the user poses related to our wine business.
You will provide helpful response to user's question and thus keeping the interaction natural and engaging.
Please respond to the following question.
"""

prompt_template = lambda question: f'''[INST] {intstructions_string} \n{question} \n[/INST]'''

question = "Tell me about white wine?"

prompt = prompt_template(question)
print(prompt)

[INST] You are a helpful and smart chatbot designed to assist customers of our wine-selling business by providing quick, accurate, and relevant answers to user's query.Your primary goal is to help users find information about our wine collection, make recommendations, answer questions about orders and shipping, and provide details on whatever question the user poses related to our wine business.
You will provide helpful response to user's question and thus keeping the interaction natural and engaging.
Please respond to the following question.
 
Tell me about white wine? 
[/INST]


In [8]:
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] You are a helpful and smart chatbot designed to assist customers of our wine-selling business by providing quick, accurate, and relevant answers to user's query.Your primary goal is to help users find information about our wine collection, make recommendations, answer questions about orders and shipping, and provide details on whatever question the user poses related to our wine business.
You will provide helpful response to user's question and thus keeping the interaction natural and engaging.
Please respond to the following question.
 
Tell me about white wine? 
[/INST] White wine is a type of wine that is produced from grapes with green or yellow skin. The color of the wine does not necessarily indicate the presence or absence of grape skins during fermentation, but rather the absence of skin contact during the production process. White wine can be made from a wide range of grape varieties, each imparting unique flavors and aromas to the final product. Some common white w

### Prepare Model for Training

In [13]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [14]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


### Preparing Training Dataset

In [16]:
# Source directory
import shutil

source_dir = "/kaggle/input/wine-dataset"

# Destination directory
destination_dir = "/kaggle/working/wine-dataset"

shutil.copytree(source_dir, destination_dir)

FileExistsError: [Errno 17] File exists: '/kaggle/working/wine-dataset'

In [17]:
from datasets import load_from_disk
data = load_from_disk("/kaggle/working/wine-dataset/dataset/wine_data.hf")
data

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 36
    })
    test: Dataset({
        features: ['example'],
        num_rows: 7
    })
})

In [18]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["example"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [19]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


### Fine-tuning Model

In [20]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "shawgpt-ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)

# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,3.395800,2.835273
1,3.247700,2.626113
2,2.967800,2.427320
4,2.523200,2.081233
5,2.398100,1.994352
6,2.304500,1.929298
8,1.963400,1.849343


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

### Push model to hub

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [ ]:
# hf_name = 'shawhin' # your hf username or org name
# model_id = hf_name + "/" + "shawgpt-ft"

In [ ]:
# model.push_to_hub(model_id)
# trainer.push_to_hub(model_id)

### Load Fine-tuned Model

In [ ]:
# # load model from hub
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM

# model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
# model = AutoModelForCausalLM.from_pretrained(model_name,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

# config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
# model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

# # load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

### Use Fine-tuned Model

In [22]:
intstructions_string = f"""You are a helpful and smart chatbot designed to assist customers of our wine-selling business by providing quick, accurate, and relevant answers to user's query.\
Your primary goal is to help users find information about our wine collection, make recommendations, answer questions about orders and shipping, and provide details on whatever question the user poses related to our wine business.
You will provide helpful response to user's question and thus keeping the interaction natural and engaging.
Please respond to the following question.
"""

prompt_template = lambda question: f'''[INST] {intstructions_string} \n{question} \n[/INST]'''

question = "Tell me about white wine?"

prompt = prompt_template(question)
print(prompt)

[INST] You are a helpful and smart chatbot designed to assist customers of our wine-selling business by providing quick, accurate, and relevant answers to user's query.Your primary goal is to help users find information about our wine collection, make recommendations, answer questions about orders and shipping, and provide details on whatever question the user poses related to our wine business.
You will provide helpful response to user's question and thus keeping the interaction natural and engaging.
Please respond to the following question.
 
Tell me about white wine? 
[/INST]


In [24]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] You are a helpful and smart chatbot designed to assist customers of our wine-selling business by providing quick, accurate, and relevant answers to user's query.Your primary goal is to help users find information about our wine collection, make recommendations, answer questions about orders and shipping, and provide details on whatever question the user poses related to our wine business.
You will provide helpful response to user's question and thus keeping the interaction natural and engaging.
Please respond to the following question.
 
Tell me about white wine? 
[/INST] White wine is made from green grapes, which are crushed and fermented to create the wine. The color of the grape skins determines the color of the wine. White grapes have thin skins, which do not impart color to the wine. White wine can be produced from a wide range of grape varieties, and it can be produced in a variety of styles, from crisp and refreshing to rich and full-bodied. White wine is often serve